In [ ]:
# conda env create --name ubs_dlcv python==3.12

In [ ]:
# Install or upgrade the ultralytics package from PyPI
# !pip install ultralytics #https://docs.ultralytics.com/quickstart/
# !pip install numpy # not working in conda env
# !conda install numpy -y
# %pip install pylabel
# %pip install ultralytics

In [ ]:
from ultralytics import YOLO
import numpy as np
# import pylabel
import os

## convert annotations from the COCO format to YOLO 

In [ ]:
# import os
# import json
# from ultralytics.data.converter import convert_coco

# dataset_root = "dataset/SARscope"  # your source dataset
# output_root = "dataset/yolo"       # YOLO-ready output
# splits = ["train", "valid", "test"]
# use_keypoints = False

# images_root = os.path.join(output_root, "images")
# labels_root = os.path.join(output_root, "labels")

# os.makedirs(images_root, exist_ok=True)
# os.makedirs(labels_root, exist_ok=True)

# # Step 1: Convert COCO JSON -> YOLO labels
# for split in splits:
#     json_file = os.path.join(dataset_root, split, "_annotations.coco.json")
#     save_dir = os.path.join(output_root, split + "_tmp")  # temporary folder
    
#     os.makedirs(save_dir, exist_ok=True)
    
#     # Convert COCO to YOLO
#     convert_coco(
#         labels_dir=json_file,
#         save_dir=save_dir,
#         use_keypoints=use_keypoints
#     )
    
#     # Move images
#     possible_img_dir = os.path.join(save_dir, "images")
#     src_img_dir = possible_img_dir if os.path.exists(possible_img_dir) else save_dir
#     dest_img_dir = os.path.join(images_root, split)
#     os.makedirs(dest_img_dir, exist_ok=True)
#     for f in os.listdir(src_img_dir):
#         if f.lower().endswith((".jpg", ".jpeg", ".png")):
#             os.rename(os.path.join(src_img_dir, f), os.path.join(dest_img_dir, f))
    
#     # Move labels if they exist
#     src_lbl_dir = os.path.join(save_dir, "labels")
#     if os.path.exists(src_lbl_dir):
#         dest_lbl_dir = os.path.join(labels_root, split)
#         os.makedirs(dest_lbl_dir, exist_ok=True)
#         for f in os.listdir(src_lbl_dir):
#             if f.endswith(".txt"):
#                 os.rename(os.path.join(src_lbl_dir, f), os.path.join(dest_lbl_dir, f))
#         os.rmdir(src_lbl_dir)
    
#     # Remove temporary folder if empty
#     if os.path.exists(possible_img_dir):
#         try:
#             os.rmdir(possible_img_dir)
#         except OSError:
#             pass
#     try:
#         os.rmdir(save_dir)
#     except OSError:
#         pass

# # Step 2: Generate dataset.yaml
# with open(os.path.join(dataset_root, "train", "_annotations.coco.json")) as f:
#     data = json.load(f)

# categories = data['categories']
# names = {i: cat['name'] for i, cat in enumerate(categories)}

# yaml_lines = [
#     f"path: {output_root}",
#     "train: images/train",
#     "val: images/valid",
#     "test: images/test",
#     "names:"
# ]

# for idx, name in names.items():
#     yaml_lines.append(f"  {idx}: {name}")

# yaml_path = os.path.join(output_root, "dataset.yaml")
# with open(yaml_path, "w") as f:
#     f.write("\n".join(yaml_lines))

# print(f"YOLOv8-ready dataset prepared at: {output_root}")
# print(f"dataset.yaml generated at: {yaml_path}")


In [ ]:
import os
import json
import shutil
from pylabel import importer, exporter
import glob

dataset_root = "dataset/SARscope"
output_root = "dataset/yolo"

if os.path.exists(output_root):
    shutil.rmtree(output_root)
    print(f"Deleted existing folder: {output_root}")
    
splits = ["train", "valid", "test"]

images_root = os.path.join(output_root, "images")
labels_root = os.path.join(output_root, "labels")

for split in splits:
    os.makedirs(os.path.join(images_root,split), exist_ok=True)
    os.makedirs(os.path.join(labels_root, split), exist_ok=True)

    print(f"Processing split: {split}")
    json_file = os.path.join(dataset_root, split, "_annotations.coco.json")
    images_dir = os.path.join(dataset_root, split)

    # Import dataset JSON
    dataset = importer.ImportCoco(
        path=json_file,
        path_to_images=images_dir
    )

    # Export YOLO labels and copy images
    # temp_out = os.path.join(output_root, split)
    labels_out = os.path.join(labels_root, split)

    dataset.export.ExportToYoloV5(labels_out)
    
    # for fname in os.listdir(temp_out):
    #     shutil.move(os.path.join(temp_out, fname),
    #                 os.path.join(labels_root, split, fname))
        
    for img_path in glob.glob(os.path.join(images_dir, "*.jpg")):
        fname = os.path.basename(img_path)  # get filename only
        shutil.copy(img_path, os.path.join(images_root, split, fname))

    # Remove the temporary folder
    # shutil.rmtree(temp_out)

# Create dataset.yaml
with open(os.path.join(dataset_root, "train", "_annotations.coco.json")) as f:
    data = json.load(f)

categories = data["categories"]
names_dict = {i: c["name"] for i, c in enumerate(categories)}

yaml_text = [
    f"path: {output_root}",
    "train: images/train",
    "val: images/valid",
    "test: images/test",
    "names:"
]
for idx, nm in names_dict.items():
    yaml_text.append(f"  {idx}: {nm}")

with open(os.path.join(output_root, "dataset.yaml"), "w") as f:
    f.write("\n".join(yaml_text))


Deleted existing folder: dataset/yolo
Processing split: train


/Users/rabinatwayana/miniforge3/envs/ubs_dlcv/lib/python3.12/site-packages/pylabel/importer.py:127: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("", inplace=True)
/Users/rabinatwayana/miniforge3/envs/ubs_dlcv/lib/python3.12/site-packages/pylabel/exporter.py:551: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yolo_dataset.ann_bbox_xmin = yolo_dataset.ann_bbox_xmin.replace(
/Users/rabinatwayana/miniforge3/envs/ubs_dlcv/lib/python3.12/site-packages/pylabel/exporter.py:554: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To 

Processing split: valid


Exporting YOLO files...: 100%|██████████| 1346/1346 [00:01<00:00, 1253.47it/s]


Processing split: test


Exporting YOLO files...: 100%|██████████| 672/672 [00:00<00:00, 1206.61it/s]
